<a href="https://colab.research.google.com/github/mbc2009/Lmp_ML/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Enviornment Prepare

## 0.0.Download \& Import

In [21]:
%%capture
%%bash
# install package
pip install opencv-python pillow
pip install segmentation_models_pytorch
pip install kaggle
pip install dropbox
pip install scikit-image

# remove unnecessary
cd /content
rm -rf *

# from Git-Hub reciprotory
git clone https://github.com/mbc2009/Lmp_ML
rm -rf /content/Lmp_ML/main.ipynb

In [26]:
# basic import
import  os, sys, time, math, random, math, shutil, pickle
from    datetime                import  datetime
from    typing                  import  List, Tuple
from    dropbox                 import  Dropbox
from    tqdm                    import  tqdm
from    matplotlib              import  pyplot      as plt
import  numpy                                       as np
import  pandas                                      as pd
import  zipfile
import  warnings
import  shutil

# advance import
from    skimage                 import  io
from    scipy                   import  interpolate
from    scipy.interpolate       import  RegularGridInterpolator
from    scipy.ndimage           import  generic_filter
from    PIL                     import  Image

# torch import
import  torch
from    torch                   import  nn          as      nn
from    torch.nn                import  functional  as F
import  torch.optim                                 as optim
from    torch.utils.data        import  Dataset, DataLoader, TensorDataset, random_split, Subset
from    torchvision             import  transforms, models
from    torchvision.transforms  import  *
import  torchvision.transforms.functional as TF
import  kagglehub

## 0.1.Functions define

### 0.1.1.File Reader

In [45]:
# fetch data from EXCEL
def fetch(len_i=10,
          sigma_i=14,
          temp_i=373,
          item_i='flux\n(L/m^2/h)',
          original_file_path='../../Database/LmpGP.xlsx'
          ):
    # 加载Excel文件
    df = pd.read_excel(original_file_path)
    # 查找行号
    row_number = df[
        (df['len\n(A)'] == len_i) &
        (df['sigma\n(A)'] == sigma_i) &
        (df['temp\n(k)'] == temp_i)
    ].index
    # 读取文件
    return df.at[row_number[0], item_i]
    # 项目名称对照表
    '''
        def __init__(self,):
            self.label_dict = {
                            "pore_radius\n(A)":                                                                                                            (r"Pore diameter ${D}_{\mathrm{GA}}$", r" $\mathrm{(nm)}$"),
                            "pore_radius\n(A)(smoothed_by_Sigma)":                                                                                         (r"Pore diameter ${D}_{\mathrm{GA}}$", r" $\mathrm{(nm)}$"),
                            "thickness_of_box\n(A)":                                                                                                       (r"GP thickness $t_{\mathrm{GA}}$", r" $\mathrm{ (\AA)}$"),
                            "bond_density\n(unitless)":                                                                                                    (r"$C-C$ bond density ${\rho}_{C-C}$", r" (unitless)"),
                            "specific_surface_area\n(m^2/g)":                                                                                              (r"Specific surface area $S_{\mathrm{GA}}$", r" $\mathrm{(m^{2}/g)}$"),
                            "specific_surface_area\n(m^2/g)(smoothed_by_Pore radius)":                                                                     (r"Specific surface area $S_{\mathrm{GA}}$", r" $\mathrm{(m^{2}/g)}$"),
                            "surface_area\n(A^2)":                                                                                                         (r"Surface area $A_{\mathrm{GA}}$", r" $\mathrm{({\AA}^{2})}$"),
                            "porosity\n(unitless)":                                                                                                        (r"Porosity $\phi_{\mathrm{GA}}$", r" ($\%$)"),
                            "porosity\n(unitless)(smoothed_by_Pore radius)":                                                                               (r"Porosity $\phi_{\mathrm{GA}}$", r" ($\%$)"),
                            "density\n(g/cm^3)":                                                                                                           (r"Density ${\rho}_{\mathrm{GA}}$",r" $\mathrm{ (mg/{cm}^{3})}$"),
                            'sigma\n(A)':                                                                                                                  (r"Sigma $\sigma$",r" $\mathrm{ (\AA)}$"),
                            'Diffusivity_EA_filtrates\n_in_membrane\n(m^2/s)':                                                                             (r"Diffusivity $\mathcal{D}$",r" $\mathrm{ (m^{2}/s)}$"),
                            'Diffusivity_TA_filtrates\n_in_membrane\n(m^2/s)':                                                                             (r"Diffusivity $\mathcal{D}$",r" $\mathrm{ (m^{2}/s)}$"),
                            'Diffusivity_TA_filtrates\n_in_membrane\n(m^2/s)(averaged)':                                                                   (r"Diffusivity $\mathcal{D}$",r" $\mathrm{ (m^{2}/s)}$"),
                            'Diffusivity_TA_filtrates\n_in_membrane\n(m^2/s)(averaged)(smoothed_by_Surface area)':                                         (r"Diffusivity $\mathcal{D}$",r" $\mathrm{ (m^{2}/s)}$"),
                            'Diffusivity_TA_filtrates\n_in_membrane\n(m^2/s)(averaged)(smoothed_by_Surface area)(smoothed_by_Pore radius)':                (r"Diffusivity $\mathcal{D}$",r" $\mathrm{ (m^{2}/s)}$"),
                            'Diffusivity_Knudsen_PoreRadius\n(m^2/s)':                                                                                     (r"Knudsen diffusivity $\mathcalD}_{Kn}$",r" $\mathrm{ (m^{2}/s)}$"),
                            'Diffusivity_Knudsen_PoreRadius\n(m^2/s)Diffusivity_Knudsen_PoreRadius\n(m^2/s)(smoothed_by_Pore radius)':                     (r"Knudsen diffusivity $\mathcalD}_{Kn}$",r" $\mathrm{ (m^{2}/s)}$"),
                            'flux\n(L/m^2/h)':                                                                                                             (r"Flux $\mathcal{J}$",r" $\mathrm{(L/m^{2}/h)}$"),
                            'flux\n(L/m^2/h)(smoothed_by_Sigma)':                                                                                          (r"Flux $\mathcal{J}$",r" $\mathrm{(L/m^{2}/h)}$"),
                            'flux\n(L/m^2/h)(smoothed_by_Pore radius)':                                                                                    (r"Flux $\mathcal{J}$",r" $\mathrm{(L/m^{2}/h)}$"),
                            'flux\n(L/m^2/h)(smoothed_by_Pore radius)(smoothed_by_Tortuosity)':                                                            (r"Flux $\mathcal{J}$",r" $\mathrm{ (L/m^{2}/h)}$"),
                            'flux\n(L/m^2/h)(smoothed_by_Pore radius)(smoothed_by_Tortuosity)(smoothed_by_Porosity)':                                      (r"Flux $\mathcal{J}$",r" $\mathrm{ (L/m^{2}/h)}$"),
                            'tortuosity\n(unitless)':                                                                                                      (r"Tortuosity ${\tau}_{\mathrm{GA}}$", r" (unitless)"),
                            'tortuosity\n(unitless)(smoothed_by_Pore radius)':                                                                             (r"Tortuosity ${\tau}_{\mathrm{GA}}$", r" (unitless)"),
                            'tortuosity\n(unitless)(smoothed_by_Surface area)':                                                                            (r"Tortuosity ${\tau}_{\mathrm{GA}}$", r" (unitless)"),
                            'CN\n_cutoff':                                                                                                                 (r"$\#$ of $C$ atoms nearby $n_{C}$", r" (unitless)"),
                            'CN\n_1/2_peak':                                                                                                               (r"$\#$ of $C$ atoms nearby $n_{C}$", r" (unitless)"),
                            'CN\n_cutoff(smoothed_by_Pore radius)':                                                                                        (r"$\#$ of $C$ atoms nearby $n_{C}$", r" (unitless)"),
                            }
            self.color_lsit()
    '''


# read the data fule
class LmpGP_file_reader():
    def __init__(self, len: int, sigma: int, temp: int, path="../../Database"):
        ## terminology of parameters
        self.sigma = sigma
        self.len   = len
        self.temp  = temp
        ## working path
        self.path           =  path
        self.As_path        =  f"{self.path}/assemblys_pretreated"
        self.Di_path        =  f"{self.path}/assemblys_distillated/len_{self.len}"
        self.folder         =  f"len_{self.len}_sigma_{self.sigma}_{self.temp}"
        self.dump_file      =  f"dump_di"
        self.data_file      =  f"data.3_len_{self.len}_sigma_{self.sigma}_pretreated"
        self.temp_dump_file =  f"./merged_dump_file.txt"
        ## auto excecuting
        #self.process_AGMD_dump_file()
        self.read_GP_data_file()
    def read_GP_data_file(self):
        ### initialize
        self.GP_atom_count, self.GP_box_bounds_info, self.GP_atom_data = None, {}, []
        file_path = os.path.join(f"{self.As_path}/{self.data_file}")
        atom_section = False

        ### extract info
        with open(file_path, 'r') as file:
            for line in file:

                ## Get the number of atoms
                if line.strip().endswith("atoms"):
                    self.GP_atom_count = int(line.split()[0])

                ## Get the box info
                # boudnds
                if 'xlo xhi' in line:
                    self.GP_box_bounds_info['x'] = list(map(float, line.strip().split()[:2]))
                if 'ylo yhi' in line:
                    self.GP_box_bounds_info['y'] = list(map(float, line.strip().split()[:2]))
                if 'zlo zhi' in line:
                    self.GP_box_bounds_info['z'] = list(map(float, line.strip().split()[:2]))

                ## Read atom data
                # Check title
                if 'Atoms' in line and '#' in line:
                    atom_section = True
                    continue
                # read
                if atom_section and line.strip():
                    parts = line.split()
                    if len(parts) >= 6:  # valid line Ensured
                        atom_id, atom_type, x, y, z = int(parts[0]), int(parts[1]), float(parts[4]), float(parts[5]), float(parts[6])
                        self.GP_atom_data.append([atom_id, atom_type, x, y, z])

            # calculate side length
            x_length = float(self.GP_box_bounds_info['x'][1]) - float(self.GP_box_bounds_info['x'][0])
            y_length = float(self.GP_box_bounds_info['y'][1]) - float(self.GP_box_bounds_info['y'][0])
            z_length = float(self.GP_box_bounds_info['z'][1]) - float(self.GP_box_bounds_info['z'][0])
            self.GP_box_lengths_info = np.array([x_length, y_length, z_length])

        ### post-treatment
        ## convert
        self.GP_atom_data = np.array(self.GP_atom_data)
        ## sorting
        # 首先按照 type 排序，然后在 type 相同的情况下按 ID 排序
        if hasattr(self, 'GP_atom_data') and self.GP_atom_data.size > 0:
            # 获取排序后的索引
            sorted_indices = np.lexsort((self.GP_atom_data[:, 0], self.GP_atom_data[:, 1]))  # 先按 ID，再按 type 排序
            # 应用排序
            self.GP_atom_data = self.GP_atom_data[sorted_indices]

    def process_AGMD_dump_file(self):
        ## Find dump file
        path = f"{self.Di_path}/{self.folder}"
        # list all files
        file_list = os.listdir(path)
        # open folders
        file_list = [f for f in file_list if os.path.isfile(os.path.join(path, f))]
        # select dump file only
        file_list = [f for f in file_list if self.dump_file in f]
        # sort by time label
        file_list = [f.replace(f"{self.dump_file}_", "") for f in file_list]
        file_list.sort()
        file_list = [f"{path}/{self.dump_file}_{f}" for f in file_list]

        ## Combine dump files into one
        output_file_path = os.path.join("./merged_dump_file.txt")
        with open(output_file_path, 'w') as output_file:
            for file_name in file_list:
                with open(os.path.join(file_name), 'r') as input_file:
                    output_file.write(input_file.read())

        ### Collect basic info
        ## Comput TIMESTEP positions list
        with open(self.temp_dump_file, 'r') as file:
            for line in file:
                ## box info
                if 'ITEM: BOX BOUNDS' in line:
                    self.box_bounds_info = {}
                    self.box_bounds_info["x"] = list(map(float, next(file).strip().split()))
                    self.box_bounds_info["y"] = list(map(float, next(file).strip().split()))
                    self.box_bounds_info["z"] = list(map(float, next(file).strip().split()))
                    x_length = float(self.box_bounds_info['x'][1]) - float(self.box_bounds_info['x'][0])
                    y_length = float(self.box_bounds_info['y'][1]) - float(self.box_bounds_info['y'][0])
                    z_length = float(self.box_bounds_info['z'][1]) - float(self.box_bounds_info['z'][0])
                    self.box_lengths_info = np.array([x_length, y_length, z_length])
                    break
        ## build timesteps list
        self.timestep_positions, self.timestep_list = {}, []
        timestep_checker = 200000
        with open(self.temp_dump_file, 'r') as file:
            line_index = -1
            while True:
                line = file.readline()
                line_index += 1
                if not line:
                    break  # 到达文件末尾
                if 'ITEM: TIMESTEP' in line:
                    # 读取时间步的值
                    timestep = int(file.readline().strip())
                    line_index += 1
                    if timestep != timestep_checker:
                        # 记录当前时间步的起始行号
                        self.timestep_positions[timestep] = line_index
                        # 将步数记入列表
                        self.timestep_list.append(timestep)
                        timestep_checker = timestep
    def read_single_timestep_for__AGMD_dump_file(self, timestep):
        #### 打开文件并开始读取
        file_path = os.path.join(self.temp_dump_file)
        with open(file_path, 'r') as file:
            ### Jump to timestep
            file.seek(0)  # 开始时先回到文件开头
            for _ in range(self.timestep_positions[timestep]):  # 跳过line_number - 1行
                file.readline()
            ## Current number of atoms
            while True:
                line = next(file)
                if 'ITEM: NUMBER OF ATOMS' in line:
                    self.atom_count = int(next(file).strip())
                    break
            ## Skip
            while True:
                line = next(file)
                if 'ITEM: ATOMS' in line:
                    break

            ### Extract atoms info
            atom_data = []
            for _ in range(self.atom_count):
                # 读取并分割行
                atom_info_str = next(file).strip().split()
                # 分别处理不同类型的数据
                # ITEM: ATOMS id type xs ys zs
                atom_info = [int(atom_info_str[0]), int(atom_info_str[1])] + [float(x) for x in atom_info_str[2:]]
                atom_data.append(atom_info)

        ### Post treatment
        # convert to NumPy matrix
        self.atom_data = np.array(atom_data)
        ## de-normalize
        if hasattr(self, 'box_lengths_info') and self.atom_data.size > 0:
            for i in range(self.atom_data.shape[0]):
                # xs
                self.atom_data[i, 2] = self.atom_data[i, 2] * self.box_lengths_info[0] + float(self.box_bounds_info["x"][0])
                # ys
                self.atom_data[i, 3] = self.atom_data[i, 3] * self.box_lengths_info[1] + float(self.box_bounds_info["y"][0])
                # zs
                self.atom_data[i, 4] = self.atom_data[i, 4] * self.box_lengths_info[2] + float(self.box_bounds_info["z"][0])
        ## sorting
        # 首先按照 type 排序，然后在 type 相同的情况下按 ID 排序
        if hasattr(self, 'atom_data') and self.atom_data.size > 0:
            # 获取排序后的索引
            sorted_indices = np.lexsort((self.atom_data[:, 0], self.atom_data[:, 1]))  # 先按 ID，再按 type 排序
            # 应用排序
            self.atom_data = self.atom_data[sorted_indices]
    def build_dump_atoms_info_list__for__AGMD_dump_file(self):
        # 初始化一个列表来存储每个时间步的原子信息矩阵
        self.atom_data_list = []
        # 遍历每个时间步
        for timestep in self.timestep_positions.keys():
            # 读取该时间步的原子信息
            self.read_single_timestep_for__AGMD_dump_file(timestep)
            # 将原子信息矩阵添加到列表中
            self.atom_data_list.append(self.atom_data)
            print(timestep)
    #def check_atom_data_list(self):
        #self.timestep_list
        #self.box_bounds_info
    def check_atom_data_list(self, output_filename="check_atom_data_list.txt"):
        """
        将atom_data_list的内容输出为LAMMPS dump文件格式
        :param output_filename: 输出文件名
        """
        with open(output_filename, 'w') as dump_file:
            for i, atom_data in enumerate(self.atom_data_list):
                # 写入时间步
                dump_file.write("ITEM: TIMESTEP\n")
                dump_file.write(f"{self.timestep_list[i]}\n")
                # 写入原子数量
                dump_file.write("ITEM: NUMBER OF ATOMS\n")
                dump_file.write(f"{len(atom_data)}\n")
                # 写入盒子边界
                dump_file.write("ITEM: BOX BOUNDS pp pp ff\n")
                for bound in ['x', 'y', 'z']:
                    dump_file.write(f"{self.box_bounds_info[bound][0]} {self.box_bounds_info[bound][1]}\n")
                # 写入原子信息
                dump_file.write("ITEM: ATOMS id type x y z\n")
                for atom in atom_data:
                    dump_file.write(" ".join(map(str, atom)) + "\n")


### 0.1.2.Data Pretreeatment

In [40]:
class Datafile_to_grids():
    def __init__(self,
                 GP_data,      # data file 原始数据
                 grid_size,    # 网格划分的尺寸
                 limits,       # 实际的 x/y/z 边界上下限
                 limits_new,  # 需要补全到的 x/y/z 边界上下限
                 output_filename = "CHECK_DataFile"
                 ):
        self.output_filename = output_filename
        self.GP_data = GP_data
        self.grid_size = grid_size
        self.xmin, self.xmax = limits["x"]
        self.ymin, self.ymax = limits["y"]
        self.zmin, self.zmax = limits["z"]
        self.xmin_new, self.xmax_new = limits_new["x"]
        self.ymin_new, self.ymax_new = limits_new["y"]
        self.zmin_new, self.zmax_new = limits_new["z"]

        # 计算原始边界和新边界的长度
        self.xlength = self.xmax - self.xmin
        self.ylength = self.ymax - self.ymin
        self.zlength = self.zmax - self.zmin
        self.xlength_new = self.xmax_new - self.xmin_new
        self.ylength_new = self.ymax_new - self.ymin_new
        self.zlength_new = self.zmax_new - self.zmin_new

        # 自动执行
        self.GP_data_pretreat()
        self.GP_data_analysis()
        self.GP_data_aligning()
        self.GP_data_to_grid()
        self.check_atom_data_list()

    def GP_data_pretreat(self):
        self.GP_data = np.delete(self.GP_data, 1, axis=1)  # 删除 atom type 列

    def GP_data_analysis(self):
        self.GP_atom_count = self.GP_data.shape[0]

    def GP_data_aligning(self):
        '''
        对data file 进行补全
        '''
        # 计算需要复制的倍数，包括正方向和负方向
        x_repeats_pos = int(np.ceil((self.xmax_new - self.xmax) / self.xlength))
        x_repeats_neg = int(np.ceil((self.xmin - self.xmin_new) / self.xlength))
        y_repeats_pos = int(np.ceil((self.ymax_new - self.ymax) / self.ylength))
        y_repeats_neg = int(np.ceil((self.ymin - self.ymin_new) / self.ylength))
        z_repeats_pos = int(np.ceil((self.zmax_new - self.zmax) / self.zlength))
        z_repeats_neg = int(np.ceil((self.zmin - self.zmin_new) / self.zlength))

        # 扩展原子坐标，包括正方向和负方向
        extended_atoms = []
        new_atom_id = 1
        for i in range(-x_repeats_neg, x_repeats_pos + 1):
            for j in range(-y_repeats_neg, y_repeats_pos + 1):
                for k in range(-z_repeats_neg, z_repeats_pos + 1):
                    for atom in self.GP_data:
                        new_atom = atom.copy()
                        new_atom[0] = new_atom_id  # 更新原子ID
                        new_atom[1] = atom[1] + i * self.xlength
                        new_atom[2] = atom[2] + j * self.ylength
                        new_atom[3] = atom[3] + k * self.zlength
                        extended_atoms.append(new_atom)
                        new_atom_id += 1

        # 转换为numpy数组
        extended_atoms = np.array(extended_atoms)

        # 过滤掉超出新边界的原子
        extended_atoms = extended_atoms[
            (extended_atoms[:, 1] >= self.xmin_new) & (extended_atoms[:, 1] < self.xmax_new) &
            (extended_atoms[:, 2] >= self.ymin_new) & (extended_atoms[:, 2] < self.ymax_new) &
            (extended_atoms[:, 3] >= self.zmin_new) & (extended_atoms[:, 3] < self.zmax_new)
        ]

        # 更新GP_data，并重新分配ID以确保连续
        self.GP_data = extended_atoms
        self.GP_data[:, 0] = np.arange(1, self.GP_data.shape[0] + 1)
        self.GP_atom_count_extended = self.GP_data.shape[0]

    def GP_data_to_grid(self):
        '''
        将原子数据转换为网格数据
        '''
        # N 格子, N+1 个点
        x_bins = np.linspace(self.xmin_new, self.xmax_new, self.grid_size[0] + 1)
        y_bins = np.linspace(self.ymin_new, self.ymax_new, self.grid_size[1] + 1)
        z_bins = np.linspace(self.zmin_new, self.zmax_new, self.grid_size[2] + 1)

        # 生成 X * Y * Z 形状的矩阵 (grid_size=(XY,Z))
        grid_counts = np.zeros(self.grid_size)

        for atom in self.GP_data:
            x_idx = np.digitize(atom[1], x_bins) - 1    # digitize() 从1开始
            y_idx = np.digitize(atom[2], y_bins) - 1
            z_idx = np.digitize(atom[3], z_bins) - 1
            if x_idx < self.grid_size[0] and y_idx < self.grid_size[1] and z_idx < self.grid_size[2]:
                grid_counts[x_idx, y_idx, z_idx] += 1

        self.grid_counts = grid_counts

        # 打印结果以检查
        #print(f"Grid counts shape: {self.grid_counts.shape}")
        #print(self.grid_counts)

    def check_atom_data_list(self):
        """
        将atom_data_list的内容输出为LAMMPS dump文件格式
        :param output_filename: 输出文件名
        """

        with open(self.output_filename, 'w') as data_file:
            # 写入基本信息
            data_file.write("LAMMPS data file via write_data, version 2 Aug 2023, timestep = 50000, units = metal\n\n")
            data_file.write(f"{self.GP_atom_count_extended} atoms\n")
            data_file.write("1 atom types\n\n")
            data_file.write(f"{self.xmin_new} {self.xmax_new} xlo xhi\n")
            data_file.write(f"{self.ymin_new} {self.ymax_new} ylo yhi\n")
            data_file.write(f"{self.zmin_new} {self.zmax_new} zlo zhi\n\n")
            data_file.write("Masses\n\n")
            data_file.write("1 12.0107\n\n")
            data_file.write("Atoms # full\n\n")
            for atom in self.GP_data:
                atom_inserted = np.insert(atom, 1, 1)                            # 在第二项位置插入 atom type   为 1
                atom_inserted = np.insert(atom_inserted, 1, 1)                   # 在第二项位置插入 molecule ID 为 1
                atom_inserted = np.insert(atom_inserted, 3, 0)                   # 在第二项位置插入 charge      为 0
                atom_inserted = np.insert(atom_inserted, len(atom_inserted), 0)  # 在第二项位置插入 velocity X  为 0
                atom_inserted = np.insert(atom_inserted, len(atom_inserted), 0)  # 在第二项位置插入 velocity Y  为 0
                atom_inserted = np.insert(atom_inserted, len(atom_inserted), 0)  # 在第二项位置插入 velocity Z  为 0

                # 将 atom ID 和 molecule ID 转换为整数, 其他保持原有精度
                formatted_line = f"{int(atom_inserted[0])} {int(atom_inserted[1])} {int(atom_inserted[2])} {int(atom_inserted[3])} " \
                                 f"{atom_inserted[4]} {atom_inserted[5]} {atom_inserted[6]} " \
                                 f"{int(atom_inserted[7])} {int(atom_inserted[8])} {int(atom_inserted[9])}"

                data_file.write(formatted_line + "\n")


class GridDataset(Dataset):
    def __init__(self, grid_counts_dict):
        self.data = []
        self.labels = []

        for label, grid in grid_counts_dict.items():
            self.data.append(grid)
            self.labels.append(self.extract_label_from_key(label))

        # 转换为 torch.Tensor
        self.data   = torch.tensor(self.data, dtype=torch.float32).unsqueeze(1)  # 添加 channel 维度
        self.labels = torch.tensor(self.labels, dtype=torch.float32)

    def extract_label_from_key(self, key):
        # 解析出 len_i, sigma_i, temp_i
        parts = key.split('_')
        len_i = int(parts[1])
        sigma_i = int(parts[3])
        temp_i = int(parts[-1])

        # 返回两个标签
        label1 = len_i
        label2 = sigma_i
        return label1, label2


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        label1, label2 = self.labels[idx]
        labels = torch.tensor([label1, label2], dtype=torch.float32)  # 将两个标签组合成一个张量
        # 打印调试信息
        print(f"Loading data at index {idx}: data shape = {data.shape}, labels = {labels}")
        return data, labels

# 1.Prepare

## 1.1 Configuration

In [32]:
class Conf:
    '''
    Pre-defined parameters for ML tranning
    '''
    #
    path = '/content/Lmp_ML'
    path_to_Database = os.path.join(path, 'DataBase')
    path_to_GA_data_file = os.path.join(path_to_Database, 'assemblys_pretreated')

    # Data parameters
    image_folder        = os.path.join(path, 'images')
    mask_folder         = os.path.join(path, 'segmaps')
    resize              = (512, 512) # it will be very painful if the image height and width are not the same!!!!

    # Determine data cleaning directory
    patched_image_folder = os.path.join(path, 'images_with_patch')
    patched_mask_folder  = os.path.join(path, 'segmaps_with_patch')

    # Training hyperparameters
    num_epochs      = 20
    batch_size      = 16
    learning_rate   = 2e-4
    momentum        = 0.9
    weight_decay    = 1.5e-4

    # Model parameters
    num_classes = 1  # For binary segmentation
    kernel_size = 3
    stride      = 1
    padding     = 1

    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1.2.Data Pre-treatment

### 1.2.1 1D performance data

### 1.2.2 3D GA structure

#### 1.2.2.1 Collect data file info

In [46]:
# initial a main list to store all info
info_list = np.empty((0, 15))

# traverse though all data file
for len_i in range(2, 21, 1):
    for sigma_i in range(8, 19, 1):
        for temp_i in range(373, 332, -10):

            # read data file info
            GP = LmpGP_file_reader(len=len_i, sigma=sigma_i, temp=temp_i, path=Conf.path_to_Database)
            GP_atom_count = GP.GP_atom_count
            GP_box_bounds_info = GP.GP_box_bounds_info
            GP_atom_data = GP.GP_atom_data

            # cal. max boundary len
            x_length = GP_box_bounds_info['x'][1] - GP_box_bounds_info['x'][0]
            y_length = GP_box_bounds_info['y'][1] - GP_box_bounds_info['y'][0]
            z_length = GP_box_bounds_info['z'][1] - GP_box_bounds_info['z'][0]
            max_BoundLength_i = np.amax([x_length, y_length, z_length])

            # cal. min boundary len
            min_Bound_i = np.amin([
                                    GP_box_bounds_info['x'][0],
                                    GP_box_bounds_info['x'][1],
                                    GP_box_bounds_info['y'][0],
                                    GP_box_bounds_info['y'][1],
                                    GP_box_bounds_info['z'][0],
                                    GP_box_bounds_info['z'][1]
                                ])

            # temp list sotring all info
            info_i = [int(len_i),                      # 0
                      int(sigma_i),                    # 1
                      int(temp_i),                     # 2
                      GP_atom_count,                   # 3
                      GP_box_bounds_info['x'][0],      # 4
                      GP_box_bounds_info['x'][1],      # 5
                      GP_box_bounds_info['y'][0],      # 6
                      GP_box_bounds_info['y'][1],      # 7
                      GP_box_bounds_info['z'][0],      # 8
                      GP_box_bounds_info['z'][1],      # 9
                      min_Bound_i,                     # 10
                      x_length,                        # 11
                      y_length,                        # 12
                      z_length,                        # 13
                      max_BoundLength_i                # 14
                    ]

            info_list = np.vstack((info_list, info_i))